In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
# 문자열을 리스트로 변환
def str_to_list(x):
    try:
        return literal_eval(x)
    except: #해당 값이 null값이거나 오류가 있을 때, None을 return 하기
        return None
    
# 딕셔너리에서 키값만 반환
def dic_to_list(x):
    try:
        temp = []
        for i in x:
            temp.append(i["key"])
        if not temp:
            return None
        else:
            return temp
    except: #해당 값이 null값이거나 오류가 있을 때, None을 return 하기
        return None

In [ ]:
problems = pd.read_csv('./data/problems.csv', encoding='utf-8')
users = pd.read_csv('./data/users.csv', encoding='utf-8')
solved_problems = pd.read_csv('./data/solved_problems.csv')

problems['tags'] = problems['tags'].apply(lambda x: str_to_list(x)) 
solved_problems['solved_problem'] = solved_problems['solved_problem'].apply(lambda x: str_to_list(x)) 
problems['tags'] = problems['tags'].apply(lambda x: dic_to_list(x)) 

In [ ]:
users_drop = ['bio', 'badgeId', 'backgroundId', 'profileImageUrl', 'voteCount', 'classDecoration', 'ratingByVoteCount',
             'coins', 'stardusts', 'joinedAt', 'bannedUntil', 'proUntil', 'isRival', 'isReverseRival']
users = users.drop(columns=users_drop, axis=1)
users

In [ ]:
problems_drop = ['titles', 'isPartial', 'votedUserCount', 'sprout', 
                'givesNoRating', 'givesNoRating', 'metadata', 'isLevelLocked']
problems = problems.drop(columns=problems_drop, axis=1)
problems

In [ ]:
ban_users = users[(users.solvedCount == 0) & ((users.rating !=0)|(users.exp !=0))].index
users = users.drop(index=ban_users).reset_index(drop=True)
users

In [ ]:
problems = problems.drop(index = problems.loc[problems['isSolvable'] == False].index).reset_index(drop=True)
problems

In [ ]:
problems = problems.drop(index = problems.loc[problems['official'] == False].index).reset_index(drop=True)
problems

In [ ]:
gudegi = [24900, 24901, 24902, 24903, 24904, 24905, 24906, 24907, 24908, 24909, 24910, 24911, 
          21292, 21293, 21294, 21295, 21296, 21297, 21298, 21299, 
           18821, 18822, 18823, 18824, 18825, 18826, 18827, 18828, 18829, 18830, 18831, 18832, 18833, 18834, 18835, 18836, 
            17106, 17107, 17108, 17109, 17110, 17111, 17112, 17113, 17114, 17115, 17116, 17117, 17118, 17119, 17120, 
           15629, 15630, 15631, 15632, 15633, 15634, 15635, 15636, 15637, 15638, 15639, 15640, 15641, 15642, 15643]

In [ ]:
problems = problems[~problems['problemId'].isin(gudegi)].reset_index(drop=True)
problems